In [3]:
import pygame
import math
import random
import time

pygame.init()

# FOR TUNING
PREDICTION_HORIZON = 60
SAFE_CHECK_HORIZON = PREDICTION_HORIZON // 2
VIS_PRED_STEPS = 40

# GRID
GRID_SPACING = 40
GRID_MARGIN = 0
GRID_EVAL_EVERY_N = 6
RECOMPUTE_MOVE_THRESH = 2.0
w, h = 800, 600
BASIN_ALPHA = 120

# SPEEDS
FPS = 120
evader_speed = 6.0
pursuer_speed = 3.0
rel_speed = evader_speed / pursuer_speed
if rel_speed <= 3.0:
    alpha_avoidance = 0.99
else:
    alpha_avoidance = 1
tangent_offset = math.pi / 6  # tangent evasion, nominal controller
print(tangent_offset)

# GOALS
evader_radius, pursuer_radius, goal_radius = 15, 15, 20
max_goals = 10
MIN_GOAL_DIST_FROM_PURSUER = 100  # Minimum distance goal must be from pursuer

# CANDIDATES
NUM_CANDIDATES = 12
SAFE_MARGIN = 2.0
BUFFER_DIST = 10  # Early-warning safety buffer
unsafe_frames = 0

# COLORS
WHITE = (255, 255, 255)
BLUE = (50, 150, 255)
RED = (255, 80, 80)
GREEN = (50, 200, 50)
BLACK = (0, 0, 0)
PURSUER2_COLOR = (180, 0, 180)  # purple
PRED_E_COLOR = (0, 120, 255)
PRED_P_COLOR = (220, 30, 30)
BASIN_COLOR = (255, 80, 80, BASIN_ALPHA)

# initial state
evader_x = w // 4
evader_y = h // 2
pursuer_x = 3 * w // 4
pursuer_y = h // 2

# Black Swan: second pursuer appears after a random goal is reached
ENABLE_BLACK_SWAN = True
black_swan_goal_idx = random.randint(1, max_goals - 1)  # trigger after reaching this goal count
pursuer2_active = False
pursuer2_x, pursuer2_y = None, None

goal_counter = 0
goal_times = []

# SCREEN
screen = pygame.display.set_mode((w, h))
pygame.display.set_caption("Black Swan Switching Filter (2 Pursuers)")
clock = pygame.time.Clock()


# SPAWN GOALS STRATEGICALLY
def spawn_goal_away_from_pursuer(pux, puy):
    max_attempts = 100
    for _ in range(max_attempts):
        gx = random.randint(0, w)
        gy = random.randint(0, h)
        dist = math.hypot(gx - pux, gy - puy)
        if dist >= MIN_GOAL_DIST_FROM_PURSUER:
            return gx, gy
    return random.randint(0, w), random.randint(0, h)


def spawn_pursuer_away_from_evader(evx, evy, min_dist=220):
    max_attempts = 100
    for _ in range(max_attempts):
        x = random.randint(0, w)
        y = random.randint(0, h)
        if math.hypot(x - evx, y - evy) >= min_dist:
            return x, y
    return random.randint(0, w), random.randint(0, h)


# Spawn the initial goal
goal_x, goal_y = spawn_goal_away_from_pursuer(pursuer_x, pursuer_y)
goal_spawn_time = pygame.time.get_ticks() / 1000.0

# Start the game
running = True
game_over = False
frame_count = 0
_last_pursuer_pos = (pursuer_x, pursuer_y)
choice_idx = 0


########################################################
# IMMEDIATE EARLY WARNING (multi-pursuer)
########################################################
def early_warning(evx, evy, pursuers):
    for px, py in pursuers:
        dist = math.hypot(evx - px, evy - py)
        if dist < (evader_radius + pursuer_radius + BUFFER_DIST):
            return True
    return False


########################################################
# PREDICTED PATHS (visualize using pursuer 1 only)
########################################################
def get_predicted_paths(evx, evy, pux, puy, mode="SAFETY", steps=VIS_PRED_STEPS):
    e_x, e_y = float(evx), float(evy)
    p_x, p_y = float(pux), float(puy)
    e_path, p_path = [], []
    for _ in range(steps):
        dx = e_x - p_x
        dy = e_y - p_y
        dist = math.hypot(dx, dy)
        if mode == "GOAL":
            gdx = goal_x - e_x
            gdy = goal_y - e_y
            gdist = math.hypot(gdx, gdy)
            if gdist > 1e-6:
                ev_vx = (gdx / gdist) * evader_speed
                ev_vy = (gdy / gdist) * evader_speed
            else:
                ev_vx, ev_vy = 0.0, 0.0
        else:
            if dist > 1e-6:
                ev_vx = (dx / dist) * evader_speed
                ev_vy = (dy / dist) * evader_speed
            else:
                ev_vx, ev_vy = 0.0, 0.0

        e_x += ev_vx
        e_y += ev_vy

        lookahead = 3.0
        pred_ex = e_x + ev_vx * lookahead
        pred_ey = e_y + ev_vy * lookahead
        pdx = pred_ex - p_x
        pdy = pred_ey - p_y
        pdist = math.hypot(pdx, pdy)
        if pdist > 1e-6:
            p_x += (pdx / pdist) * pursuer_speed
            p_y += (pdy / pdist) * pursuer_speed

        e_x = max(evader_radius, min(w - evader_radius, e_x))
        e_y = max(evader_radius, min(h - evader_radius, e_y))
        p_x = max(pursuer_radius, min(w - pursuer_radius, p_x))
        p_y = max(pursuer_radius, min(h - pursuer_radius, p_y))

        e_path.append((e_x, e_y))
        p_path.append((p_x, p_y))

        if math.hypot(e_x - p_x, e_y - p_y) <= evader_radius + pursuer_radius:
            break
    return e_path, p_path


########################################################
# SHORT_TERM CAPTURE SIM DETAILS (single pursuer)
########################################################
def will_be_captured(evx, evy, pux, puy, horizon=PREDICTION_HORIZON):
    e_x, e_y = float(evx), float(evy)
    p_x, p_y = float(pux), float(puy)
    for _ in range(horizon):
        dx = e_x - p_x
        dy = e_y - p_y
        dist = math.hypot(dx, dy)
        if dist <= (evader_radius + pursuer_radius):
            return True

        if dist > 1e-6:
            ev_vx = (dx / dist) * evader_speed
            ev_vy = (dy / dist) * evader_speed
        else:
            ev_vx, ev_vy = 0.0, 0.0

        e_x += ev_vx
        e_y += ev_vy

        e_x = max(evader_radius, min(w - evader_radius, e_x))
        e_y = max(evader_radius, min(h - evader_radius, e_y))

        lookahead = 3.0
        pred_ex = e_x + ev_vx * lookahead
        pred_ey = e_y + ev_vy * lookahead

        pdx = pred_ex - p_x
        pdy = pred_ey - p_y
        pdist = math.hypot(pdx, pdy)
        if pdist > 1e-6:
            p_x += (pdx / pdist) * pursuer_speed
            p_y += (pdy / pdist) * pursuer_speed  # NOTE: this was missing in many buggy variants

        p_x = max(pursuer_radius, min(w - pursuer_radius, p_x))
        p_y = max(pursuer_radius, min(h - pursuer_radius, p_y))

        if math.hypot(e_x - p_x, e_y - p_y) <= evader_radius + pursuer_radius:
            return True
    return False


def will_be_captured_multi(evx, evy, pursuers, horizon=PREDICTION_HORIZON):
    for px, py in pursuers:
        if will_be_captured(evx, evy, px, py, horizon=horizon):
            return True
    return False


########################################################
# LONG_TERM BASIN CAPTURE (computed w/ current pursuers set)
########################################################
grid_xs = list(range(GRID_MARGIN + GRID_SPACING // 2, w, GRID_SPACING))
grid_ys = list(range(GRID_MARGIN + GRID_SPACING // 2, h, GRID_SPACING))
nx = len(grid_xs)
ny = len(grid_ys)
basin_grid = [[False] * ny for _ in range(nx)]
basin_stamp = -1


def recompute_basin(pursuers):
    global basin_grid, basin_stamp
    for i, gx in enumerate(grid_xs):
        for j, gy in enumerate(grid_ys):
            basin_grid[i][j] = will_be_captured_multi(
                gx, gy, pursuers, horizon=SAFE_CHECK_HORIZON
            )
    basin_stamp = frame_count


def is_in_sampled_basin(x, y):
    i = int(round((x - (GRID_SPACING / 2)) / GRID_SPACING))
    j = int(round((y - (GRID_SPACING / 2)) / GRID_SPACING))
    i = max(0, min(nx - 1, i))
    j = max(0, min(ny - 1, j))
    return basin_grid[i][j]


########################################################
# Choose EVADER ACTION (multi-pursuer aware)
########################################################
def choose_evader_action(evx, evy, goalx, goaly, pursuers):
    gdx = goalx - evx
    gdy = goaly - evy
    gdist = math.hypot(gdx, gdy)
    goal_dir = math.atan2(gdy, gdx) if gdist > 1e-6 else 0.0

    # "away" direction: use nearest pursuer
    nearest_px, nearest_py = min(pursuers, key=lambda p: math.hypot(evx - p[0], evy - p[1]))
    pdx = evx - nearest_px
    pdy = evy - nearest_py
    pdist = math.hypot(pdx, pdy)
    away_dir = math.atan2(pdy, pdx) if pdist > 1e-6 else goal_dir

    # tangent escape directions relative to nearest pursuer (works better than using pursuer1 always)
    angle_to_nearest = math.atan2(evy - nearest_py, evx - nearest_px)
    tangent_angles = [angle_to_nearest + tangent_offset, angle_to_nearest - tangent_offset]

    candidates = [goal_dir, away_dir] + tangent_angles
    for k in range(NUM_CANDIDATES):
        frac = k / max(1, NUM_CANDIDATES - 1)
        diff = ((away_dir - goal_dir + math.pi) % (2 * math.pi)) - math.pi
        candidates.append(goal_dir + diff * frac)

    safe_candidates = []
    scored_candidates = []

    for idx, angle in enumerate(candidates):
        nx_ = evx + math.cos(angle) * evader_speed
        ny_ = evy + math.sin(angle) * evader_speed
        nx_ = max(evader_radius, min(w - evader_radius, nx_))
        ny_ = max(evader_radius, min(h - evader_radius, ny_))

        # Edge-awareness: nudge angles away from walls if candidate is too close
        angle2 = angle
        if nx_ < evader_radius * 2:
            angle2 += math.radians(30)
        elif nx_ > w - evader_radius * 2:
            angle2 -= math.radians(30)
        if ny_ < evader_radius * 2:
            angle2 += math.radians(30)
        elif ny_ > h - evader_radius * 2:
            angle2 -= math.radians(30)

        nx = evx + math.cos(angle2) * evader_speed
        ny = evy + math.sin(angle2) * evader_speed
        nx = max(evader_radius, min(w - evader_radius, nx))
        ny = max(evader_radius, min(h - evader_radius, ny))

        # reject moves that reduce distance to ANY pursuer
        reject = False
        for px, py in pursuers:
            if math.hypot(nx - px, ny - py) < alpha_avoidance * math.hypot(evx - px, evy - py):
                reject = True
                break
        if reject:
            continue

        if not is_in_sampled_basin(nx, ny):
            ang_diff = abs(((angle2 - goal_dir + math.pi) % (2 * math.pi)) - math.pi)
            safe_candidates.append((ang_diff, idx, nx, ny, angle2))
        else:
            captured = will_be_captured_multi(nx, ny, pursuers, horizon=SAFE_CHECK_HORIZON)
            score = 0.0 if captured else 0.5
            scored_candidates.append((score, idx, nx, ny, angle2))

    if safe_candidates:
        safe_candidates.sort(key=lambda t: (t[0], t[1]))
        _, chosen_idx, nx, ny, _ = safe_candidates[0]
        return nx, ny, chosen_idx, True
    if scored_candidates:
        scored_candidates.sort(key=lambda t: (-t[0], t[1]))
        _, chosen_idx, nx, ny, _ = scored_candidates[0]
        return nx, ny, chosen_idx, False
    return evx, evy, 0, False


# main loop
while running:
    clock.tick(FPS)
    screen.fill(WHITE)
    frame_count += 1

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            game_over = True

    if not game_over:
        pursuers = [(pursuer_x, pursuer_y)] + ([(pursuer2_x, pursuer2_y)] if pursuer2_active else [])

        dxp = pursuer_x - _last_pursuer_pos[0]
        dyp = pursuer_y - _last_pursuer_pos[1]
        if (
            frame_count == 1
            or math.hypot(dxp, dyp) >= RECOMPUTE_MOVE_THRESH
            or frame_count % GRID_EVAL_EVERY_N == 0
        ):
            recompute_basin(pursuers)
            _last_pursuer_pos = (pursuer_x, pursuer_y)

        # choose the evader action
        next_ex, next_ey, choice_idx, step_safe = choose_evader_action(
            evader_x, evader_y, goal_x, goal_y, pursuers
        )

        # switching logic
        current_unsafe = is_in_sampled_basin(evader_x, evader_y) or early_warning(evader_x, evader_y, pursuers)
        unsafe_frames = unsafe_frames + 1 if current_unsafe else max(unsafe_frames - 1, 0)
        mode = "SAFETY" if current_unsafe else "GOAL"

        # update the evader position
        evader_x, evader_y = next_ex, next_ey

        # pursuer 1 pure pursuit
        pdx = evader_x - pursuer_x
        pdy = evader_y - pursuer_y
        pdist = math.hypot(pdx, pdy)
        if pdist > 1e-6:
            pursuer_x += (pdx / pdist) * pursuer_speed
            pursuer_y += (pdy / pdist) * pursuer_speed
        pursuer_x = max(pursuer_radius, min(w - pursuer_radius, pursuer_x))
        pursuer_y = max(pursuer_radius, min(h - pursuer_radius, pursuer_y))

        # pursuer 2 pure pursuit (if active)
        if pursuer2_active:
            dx2 = evader_x - pursuer2_x
            dy2 = evader_y - pursuer2_y
            d2 = math.hypot(dx2, dy2)
            if d2 > 1e-6:
                pursuer2_x += (dx2 / d2) * pursuer_speed
                pursuer2_y += (dy2 / d2) * pursuer_speed
            pursuer2_x = max(pursuer_radius, min(w - pursuer_radius, pursuer2_x))
            pursuer2_y = max(pursuer_radius, min(h - pursuer_radius, pursuer2_y))

        # keep evader in bounds
        evader_x = max(evader_radius, min(w - evader_radius, evader_x))
        evader_y = max(evader_radius, min(h - evader_radius, evader_y))

        # collisions (caught by ANY pursuer)
        pursuers = [(pursuer_x, pursuer_y)] + ([(pursuer2_x, pursuer2_y)] if pursuer2_active else [])
        caught = any(
            math.hypot(evader_x - px, evader_y - py) <= evader_radius + pursuer_radius
            for px, py in pursuers
        )
        if caught:
            game_over = True
            running = False

        # goals
        if goal_counter < max_goals and math.hypot(evader_x - goal_x, evader_y - goal_y) <= evader_radius + goal_radius:
            goal_times.append(pygame.time.get_ticks() / 1000.0 - goal_spawn_time)
            goal_counter += 1

            # BLACK SWAN TRIGGER: spawn pursuer 2 right after reaching the trigger goal count
            if ENABLE_BLACK_SWAN and (not pursuer2_active) and goal_counter == black_swan_goal_idx:
                pursuer2_x, pursuer2_y = spawn_pursuer_away_from_evader(evader_x, evader_y)
                pursuer2_active = True

            if goal_counter < max_goals:
                # keep spawning goal away from pursuer 1 (and pursuer 2 if active) — use the closer pursuer
                if pursuer2_active:
                    # choose nearer pursuer to keep goal from spawning right beside either
                    nearer_px, nearer_py = min(
                        [(pursuer_x, pursuer_y), (pursuer2_x, pursuer2_y)],
                        key=lambda p: math.hypot(evader_x - p[0], evader_y - p[1])
                    )
                    goal_x, goal_y = spawn_goal_away_from_pursuer(nearer_px, nearer_py)
                else:
                    goal_x, goal_y = spawn_goal_away_from_pursuer(pursuer_x, pursuer_y)
                goal_spawn_time = pygame.time.get_ticks() / 1000.0
            else:
                game_over = True
                running = False

    ####################################
    # VISUALIZATION
    ####################################
    pred_e_path, pred_p_path = get_predicted_paths(evader_x, evader_y, pursuer_x, pursuer_y, mode=mode)

    # overlay for the basin
    overlay = pygame.Surface((w, h), flags=pygame.SRCALPHA)
    overlay.fill((0, 0, 0, 0))
    for i, gx in enumerate(grid_xs):
        for j, gy in enumerate(grid_ys):
            if basin_grid[i][j]:
                left = gx - GRID_SPACING // 2
                top = gy - GRID_SPACING // 2
                rect = pygame.Rect(left, top, GRID_SPACING, GRID_SPACING)
                overlay.fill((BASIN_COLOR[0], BASIN_COLOR[1], BASIN_COLOR[2], BASIN_ALPHA), rect)
    screen.blit(overlay, (0, 0))

    # draw the predicted paths (pursuer 1 only)
    for i in range(1, len(pred_e_path)):
        pygame.draw.line(
            screen, PRED_E_COLOR,
            (int(pred_e_path[i - 1][0]), int(pred_e_path[i - 1][1])),
            (int(pred_e_path[i][0]), int(pred_e_path[i][1])), 2
        )
    for i in range(1, len(pred_p_path)):
        pygame.draw.line(
            screen, PRED_P_COLOR,
            (int(pred_p_path[i - 1][0]), int(pred_p_path[i - 1][1])),
            (int(pred_p_path[i][0]), int(pred_p_path[i][1])), 2
        )

    # draw pursuers, evader, goal
    pygame.draw.circle(screen, RED, (int(pursuer_x), int(pursuer_y)), pursuer_radius)
    if pursuer2_active:
        pygame.draw.circle(screen, PURSUER2_COLOR, (int(pursuer2_x), int(pursuer2_y)), pursuer_radius)

    pygame.draw.circle(screen, BLUE, (int(evader_x), int(evader_y)), evader_radius)
    pygame.draw.circle(screen, GREEN, (int(goal_x), int(goal_y)), goal_radius)

    # draw lines to pursuers
    pygame.draw.line(screen, BLACK, (int(evader_x), int(evader_y)), (int(pursuer_x), int(pursuer_y)), 1)
    if pursuer2_active:
        pygame.draw.line(screen, BLACK, (int(evader_x), int(evader_y)), (int(pursuer2_x), int(pursuer2_y)), 1)

    # text
    font = pygame.font.Font(None, 24)
    if not game_over:
        screen.blit(font.render(f"Goals: {goal_counter}/{max_goals}", True, GREEN), (10, 8))
        screen.blit(font.render(f"Mode: {mode}", True, RED if mode != "GOAL" else GREEN), (10, 32))
        screen.blit(font.render(f"Grid: {nx}x{ny}", True, BLACK), (10, 56))
        screen.blit(font.render(f"Choice idx: {choice_idx}", True, BLACK), (10, 80))
        screen.blit(font.render(f"Pred H: {PREDICTION_HORIZON}", True, BLACK), (10, 104))
        if ENABLE_BLACK_SWAN:
            screen.blit(
                font.render(
                    f"Black Swan trigger: goal {black_swan_goal_idx} | active: {pursuer2_active}",
                    True, PURSUER2_COLOR if pursuer2_active else BLACK
                ),
                (10, 128),
            )
    else:
        if goal_counter >= max_goals:
            msg = font.render(f"VICTORY! {goal_counter} goals", True, GREEN)
        else:
            msg = font.render("CAUGHT - game over", True, RED)
        screen.blit(msg, msg.get_rect(center=(w // 2, h // 2)))

    pygame.display.flip()

pygame.quit()


0.5235987755982988
